In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv("resale-flat-prices-2022-jan.csv")

In [ ]:
"""" 
1/ Séparation des colonnes :
numeric_columns : contient les colonnes numériques uniquement.
- non_numeric_columns : contient les colonnes non numériques (catégoriques, dates, etc.).

2/ Imputation des valeurs manquantes :

- Appliquez l'imputation avec la moyenne uniquement sur les colonnes numériques.
- Les colonnes non numériques sont conservées sans modification à ce stade.

3/ Fusion des colonnes :

Après traitement, combinez les colonnes numériques et non numériques en un seul DataFrame.""""

In [ ]:
# Traitement des valeurs manquantes

# Identifier les colonnes numériques et non numériques
numeric_columns = df.select_dtypes(include=['number']).columns
non_numeric_columns = df.select_dtypes(exclude=['number']).columns

# Appliquer SimpleImputer uniquement sur les colonnes numériques
imputer = SimpleImputer(strategy="mean")  # Remplacer les valeurs manquantes par la moyenne
df_numeric = pd.DataFrame(imputer.fit_transform(df[numeric_columns]), columns=numeric_columns)

# Conserver les colonnes non numériques telles quelles pour l'instant
df_non_numeric = df[non_numeric_columns].copy()

# Combiner les deux parties
df_imputed = pd.concat([df_numeric, df_non_numeric], axis=1)

# Vérification que les valeurs manquantes ont été traitées
print("\nNombre de valeurs manquantes après imputation:")
print(df_imputed.isnull().sum())



Nombre de valeurs manquantes après imputation:
floor_area_sqm         0
lease_commence_date    0
resale_price           0
month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
flat_model             0
remaining_lease        0
dtype: int64


In [ ]:
"""1/ Transformation des variables catégorielles :

- Encodez les colonnes catégorielles avec des techniques comme pd.get_dummies.

- Convertir les variables catégorielles en numériques permet aux algorithmes de machine learning de les interpréter.

# 2/ Transformation des colonnes de date :

Extraire des informations utiles (année, mois, jour) à partir de la colonne lease_commence_date.

# 3/ Standardisation des colonnes numériques :

- La standardisation met les valeurs des colonnes numériques à une échelle commune, en particulier pour que les grandes valeurs ne dominent pas les petites dans les calculs des modèles.

- Appliquez la standardisation (avec StandardScaler) sur les colonnes numériques pour améliorer la convergence des modèles.

- Scalage des variables numériques :

Standardiser les colonnes numériques pour les amener sur une même échelle, assurant ainsi que chaque variable contribue de manière égale à l'analyse."""

In [4]:
# 1. Vérification des colonnes disponibles
print("Colonnes du dataset :")
print(df.columns)

Colonnes du dataset :
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price'],
      dtype='object')


In [5]:
# 2. Transformation des variables catégorielles
# Sélectionner les colonnes catégorielles
categorical_columns = df_imputed.select_dtypes(include=['object']).columns.tolist()

# Affichage des colonnes catégorielles avant la transformation
print("Colonnes catégorielles avant transformation :")
print(categorical_columns)

# Exclure 'remaining_lease' car c'est une variable numérique après transformation
categorical_columns.remove('remaining_lease')

# Appliquer la transformation en variables numériques (dummy variables)
df_encoded = pd.get_dummies(df_imputed, columns=categorical_columns, drop_first=True)

# Affichage des colonnes après transformation
print("\nColonnes après transformation (dummies) :")
print(df_encoded.columns.tolist())


Colonnes catégorielles avant transformation :
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'flat_model', 'remaining_lease']

Colonnes après transformation (dummies) :
['floor_area_sqm', 'lease_commence_date', 'resale_price', 'remaining_lease', 'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK', 'town_BUKIT MERAH', 'town_BUKIT PANJANG', 'town_BUKIT TIMAH', 'town_CENTRAL AREA', 'town_CHOA CHU KANG', 'town_CLEMENTI', 'town_GEYLANG', 'town_HOUGANG', 'town_JURONG EAST', 'town_JURONG WEST', 'town_KALLANG/WHAMPOA', 'town_MARINE PARADE', 'town_PASIR RIS', 'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG', 'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS', 'town_YISHUN', 'flat_type_3 ROOM', 'flat_type_4 ROOM', 'flat_type_5 ROOM', 'flat_type_EXECUTIVE', 'flat_type_MULTI-GENERATION', 'block_10', 'block_100', 'block_101', 'block_101A', 'block_101C', 'block_101D', 'block_102', 'block_102D', 'block_103', 'block_104', 'block_104B', 'block_

In [6]:
# 3. Transformation de la colonne 'lease_commence_date'
# Affichage de la colonne 'lease_commence_date' avant transformation
print("Aperçu de la colonne 'lease_commence_date' avant transformation :")
print(df_encoded['lease_commence_date'].head())

# Conversion de la colonne 'lease_commence_date' en format datetime
df_encoded['lease_commence_date'] = pd.to_datetime(df_encoded['lease_commence_date'], errors='coerce')

# Extraction des informations de l'année, du mois et du jour
df_encoded['year'] = df_encoded['lease_commence_date'].dt.year
df_encoded['month_date'] = df_encoded['lease_commence_date'].dt.month
df_encoded['day'] = df_encoded['lease_commence_date'].dt.day

# Affichage des nouvelles colonnes extraites
print("\nAperçu des nouvelles colonnes extraites (year, month_date, day) :")
print(df_encoded[['year', 'month_date', 'day']].head())

# Suppression de la colonne 'lease_commence_date' d'origine
df_encoded = df_encoded.drop(columns=['lease_commence_date'])

# Affichage du DataFrame après suppression de la colonne 'lease_commence_date'
print("\nAperçu du DataFrame après suppression de 'lease_commence_date' :")
print(df_encoded.head())


Aperçu de la colonne 'lease_commence_date' avant transformation :
0    1977.0
1    1977.0
2    1978.0
3    1981.0
4    1980.0
Name: lease_commence_date, dtype: float64

Aperçu des nouvelles colonnes extraites (year, month_date, day) :
   year  month_date  day
0  1970           1    1
1  1970           1    1
2  1970           1    1
3  1970           1    1
4  1970           1    1

Aperçu du DataFrame après suppression de 'lease_commence_date' :
   floor_area_sqm  resale_price     remaining_lease  town_BEDOK  town_BISHAN  \
0            44.0      245000.0  54 years 05 months       False        False   
1            73.0      358000.0  54 years 05 months       False        False   
2            67.0      355000.0   55 years 01 month       False        False   
3            68.0      338000.0            58 years       False        False   
4            82.0      420000.0  57 years 02 months       False        False   

   town_BUKIT BATOK  town_BUKIT MERAH  town_BUKIT PANJANG  town_BUKI

In [7]:
# 4. Conversion de 'remaining_lease' en nombre d'années restantes
# Affichage de la colonne 'remaining_lease' avant transformation
print("Aperçu de la colonne 'remaining_lease' avant transformation :")
print(df_encoded['remaining_lease'].head())

# Extraction des années et des mois de la colonne 'remaining_lease'
df_encoded['remaining_lease_years'] = df_encoded['remaining_lease'].str.extract('(\d+)').astype(float)
df_encoded['remaining_lease_months'] = df_encoded['remaining_lease'].str.extract('(\d+)(?= years)').astype(float)

# Conversion en années restantes
df_encoded['remaining_lease_years'] = df_encoded['remaining_lease_years'] + df_encoded['remaining_lease_months'] / 12

# Affichage des nouvelles colonnes créées pour les années et mois restants
print("\nAperçu des nouvelles colonnes 'remaining_lease_years' après transformation :")
print(df_encoded[['remaining_lease_years']].head())

# Suppression de la colonne 'remaining_lease' d'origine
df_encoded = df_encoded.drop(columns=['remaining_lease'])

# Affichage du DataFrame après suppression de la colonne 'remaining_lease'
print("\nAperçu du DataFrame après suppression de 'remaining_lease' :")
print(df_encoded.head())


Aperçu de la colonne 'remaining_lease' avant transformation :
0    54 years 05 months
1    54 years 05 months
2     55 years 01 month
3              58 years
4    57 years 02 months
Name: remaining_lease, dtype: object

Aperçu des nouvelles colonnes 'remaining_lease_years' après transformation :
   remaining_lease_years
0              58.500000
1              58.500000
2              59.583333
3              62.833333
4              61.750000

Aperçu du DataFrame après suppression de 'remaining_lease' :
   floor_area_sqm  resale_price  town_BEDOK  town_BISHAN  town_BUKIT BATOK  \
0            44.0      245000.0       False        False             False   
1            73.0      358000.0       False        False             False   
2            67.0      355000.0       False        False             False   
3            68.0      338000.0       False        False             False   
4            82.0      420000.0       False        False             False   

   town_BUKIT MERAH  t

In [8]:
# 5. Normalisation des variables numériques
numerical_columns = df_encoded.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_columns.remove('resale_price')  # Exclure la cible 'resale_price'
print(f"Colonnes numériques détectées automatiquement : {numerical_columns}")

scaler = StandardScaler()
df_encoded[numerical_columns] = scaler.fit_transform(df_encoded[numerical_columns])


Colonnes numériques détectées automatiquement : ['floor_area_sqm', 'remaining_lease_years', 'remaining_lease_months']


- Le random_state=42 garantit que la séparation reste identique à chaque exécution

- Le paramètre random_state=42 garantit que la division des données entre ensemble d'entraînement et de test reste identique à chaque exécution du code, même si elle est aléatoire. Cela permet de reproduire les mêmes résultats pour des analyses ou tests comparatifs.

In [9]:
# 6. Division des données
X = df_encoded.drop(columns=['resale_price'])
y = df_encoded['resale_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Taille de l'ensemble d'entraînement : {X_train.shape[0]} échantillons")
print(f"Taille de l'ensemble de test : {X_test.shape[0]} échantillons")

# 7. Vérification finale
print("\nAperçu après traitement des données:")
print(df_encoded.head())

Taille de l'ensemble d'entraînement : 1953 échantillons
Taille de l'ensemble de test : 489 échantillons

Aperçu après traitement des données:
   floor_area_sqm  resale_price  town_BEDOK  town_BISHAN  town_BUKIT BATOK  \
0       -2.249380      245000.0       False        False             False   
1       -1.018207      358000.0       False        False             False   
2       -1.272932      355000.0       False        False             False   
3       -1.230478      338000.0       False        False             False   
4       -0.636118      420000.0       False        False             False   

   town_BUKIT MERAH  town_BUKIT PANJANG  town_BUKIT TIMAH  town_CENTRAL AREA  \
0             False               False             False              False   
1             False               False             False              False   
2             False               False             False              False   
3             False               False             False            

In [10]:
#Sauvegarder les données prétraitées dans le notebook 2
df.to_csv('preprocessed_data.csv', index=False)
